## Import Libraries

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.10.0


In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

IndexError: list index out of range

In [9]:
# Import Libraries
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from keras import layers


# text editor
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

## Load Model

In [10]:
# load model
model_load = keras.models.load_model("FINAL_LSTM.tf")

## Model Inferencing

In [11]:
#inser tweet
message = ['This is the day', 'I will not give up !', 'please, i want to talk to someone']

# Create Dataframe
tweet = pd.DataFrame()
tweet['message'] = message
tweet.head()

,message
0,This is the day
1,I will not give up !
2,"please, i want to talk to someone"


## Preprocessing

In [12]:
# Create Lemmatizer and Stopwords
stpwds_eng = list(set(stopwords.words('english')))
stpwds_eng.append(['oh','s'])

# lemmatization
nltk.download('wordnet')
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()


#Create Preprocessing Function
def pre_process(text):
 
  text = text.lower() #to lowercase
  
  text = re.sub("@[A-Za-z0-9_]+", " ", text)# Remove Mention
  
  text = re.sub("#[A-Za-z0-9_]+", " ", text)# Remove Hashtag
  
  text = re.sub(r"\\n", " ",text)# Remove \n
 
  text = text.strip() # Remove Whitespace
  
  text = re.sub(r"http\S+", " ", text) # Remove Link
  text = re.sub(r"www.\S+", " ", text)
  
  text = re.sub("[^A-Za-z\s']", " ", text)# Remove symbols, emojis

  text = re.sub(r'\b\w{2}\b', '', text)# Remove words with 2 letters or less

  tokens = word_tokenize(text)# Tokenization

  text = ' '.join([word for word in tokens if word not in stpwds_eng])# Remove Stopwords
  
  text = lemmatizer.lemmatize(text)# Lemmatizing using WordLemmatizer
  
  return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ravin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
## Preprocessing
tweet['text_processed'] = tweet['message'].apply(lambda x: pre_process(x))
tweet

,message,text_processed
0,This is the day,day
1,I will not give up !,give
2,"please, i want to talk to someone",please want talk someone


In [14]:
#Remove word with 2 letter or less
words_2 = re.compile(r'\W*\b\w{1,2}\b')

tweet['text_processed']=tweet['text_processed'].apply(lambda x: words_2.sub('', x) )
tweet.head()

,message,text_processed
0,This is the day,day
1,I will not give up !,give
2,"please, i want to talk to someone",please want talk someone


In [15]:
# Prediction
tweet_prediction = model_load.predict(tweet['text_processed'])
tweet_prediction = np.where(tweet_prediction>= 0.5,1,0)

predicted_sentiment=[]

for i in tweet_prediction:
    if i == 0:
        predicted_sentiment.append('Not Depreseed')
    else :
        predicted_sentiment.append('Depreseed')

tweet['prediction_sentiment'] = predicted_sentiment
tweet


1/1 [==============================] - 1s 521ms/step


,message,text_processed,prediction_sentiment
0,This is the day,day,Not Depreseed
1,I will not give up !,give,Not Depreseed
2,"please, i want to talk to someone",please want talk someone,Depreseed
